In [18]:
import pandas as pd
import pulp as plp
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpAffineExpression
import numpy as np


In [4]:
solver_list = pl.listSolvers(onlyAvailable=True)
solver_list

['PULP_CBC_CMD']

In [5]:
foods = pd.read_csv('./Data_formatted/nv_df.csv', index_col='Unnamed: 0')
constraints = pd.read_csv('./Data_formatted/constraints.csv', index_col = 'Unnamed: 0')

First step: create LpVariable for each food

In [6]:
foods['calories'].to_dict()

{'asparagus_cooked': 39.6,
 'avocado_cubed_raw': 240.0,
 'beet_greens_boiled': 38.88,
 'beets_sliced_cooked': 74.8,
 'bell_peppers_sliced_red_raw': 28.52,
 'bok_choy_cooked': 20.4,
 'broccoli_chopped_cooked': 54.6,
 'brussels_sprouts_cooked': 56.16,
 'cabbage_red_cooked': 43.5,
 'carrots_sliced_raw': 50.02,
 'cauliflower_cooked': 28.52,
 'celery_diced_raw': 16.16,
 'collard_greens_chopped_cooked': 62.7,
 'corn_yellow_cooked': 73.92,
 'cucumber_sliced_raw': 15.6,
 'eggplant_cubed_cooked': 34.65,
 'fennel_sliced_raw': 26.97,
 'garlic_raw': 26.82,
 'green_beans_cooked': 43.75,
 'green_peas_cooked': 115.71,
 'kale_cooked': 36.4,
 'leeks_cooked': 32.24,
 'mushrooms_crimini_raw': 15.84,
 'mushrooms_shiitake_cooked': 40.6,
 'mustard_greens_cooked': 36.4,
 'olive_oil_cold_pressed_extra_virgin': 119.34,
 'olives_black_canned': 154.56,
 'onions_chopped_cooked': 92.4,
 'potatoes_baked': 160.89,
 'romaine_lettuce_raw': 15.98,
 'sea_vegetables_dulse_dried': 10.85,
 'spinach_cooked': 41.4,
 'summer_

In [23]:
# Decision Variables
food_vars  = {food:
plp.LpVariable(cat=plp.LpContinuous, 
               lowBound=0,
               name=f"servings_{food}") 
for food in foods.index.values}

In [20]:
# Objective Function
min_cals = plp.LpProblem("Min_Calories",plp.LpMinimize)

In [25]:
 min_cals += lpSum([foods.calories[food]*[food_vars[food] \
                    for food in foods.index.values]])

NameError: name 'food' is not defined

[LpVariable](https://stackoverflow.com/questions/75710806/large-scale-lp-decision-variable-definition-pulp)

In [9]:
objective = LpAffineExpression(e=foods.calories.to_dict())

In [11]:
dir(pulp)

['CHOCO_CMD',
 'COIN',
 'COINMP_DLL',
 'COINMP_DLL_load_dll',
 'COIN_CMD',
 'CPLEX',
 'CPLEX_CMD',
 'CPLEX_PY',
 'DIRNAME',
 'EPS',
 'FixedElasticSubProblem',
 'FractionElasticSubProblem',
 'GLPK',
 'GLPK_CMD',
 'GUROBI',
 'GUROBI_CMD',
 'HiGHS_CMD',
 'Iterable',
 'LpAffineExpression',
 'LpBinary',
 'LpCategories',
 'LpConstraint',
 'LpConstraintEQ',
 'LpConstraintGE',
 'LpConstraintLE',
 'LpConstraintSenses',
 'LpConstraintTypeToMps',
 'LpConstraintVar',
 'LpContinuous',
 'LpCplexLPLineSize',
 'LpElement',
 'LpFractionConstraint',
 'LpInteger',
 'LpMaximize',
 'LpMinimize',
 'LpProblem',
 'LpSenses',
 'LpSolution',
 'LpSolutionInfeasible',
 'LpSolutionIntegerFeasible',
 'LpSolutionNoSolutionFound',
 'LpSolutionOptimal',
 'LpSolutionUnbounded',
 'LpSolver',
 'LpSolverDefault',
 'LpSolver_CMD',
 'LpStatus',
 'LpStatusInfeasible',
 'LpStatusNotSolved',
 'LpStatusOptimal',
 'LpStatusToSolution',
 'LpStatusUnbounded',
 'LpStatusUndefined',
 'LpVariable',
 'MIPCL_CMD',
 'MOSEK',
 'OrderedDi

In [12]:
LpCon

TypeError: __init__() missing 1 required positional argument: 'name'

In [64]:
usr_mass = input("How many kilos do you weigh?")
usr_mass = int(usr_mass)
constraints.iloc[-11:,0] *= usr_mass

How many kilos do you weigh? 76


In [65]:
constraints.iloc[-11:,0]

chloride                   174.8
biotin                    2280.0
chromium                  2660.0
cysteine_methionine       1444.0
histidine                 1064.0
leucine                   3192.0
lysine                    2888.0
methionine                 988.0
phenylalanine_tyrosine    2508.0
threonine                 1520.0
valine                    1824.0
Name: min, dtype: float64

In [66]:
protein_calories = foods['protein']/foods['calories'].sort_values(ascending=False).copy()

In [67]:
protein_calories.sort_values(ascending=False)

tuna_yellowfin_fillet_baked                 0.224257
cod_pacific_fillet_baked                    0.220355
turkey_pasture-raised_light_meat_roasted    0.204979
shrimp_large_steamed                        0.191418
chicken_pasture-raised_breast_roasted       0.188018
                                              ...   
cranberries_fresh_whole                     0.008478
olives_black_canned                         0.007311
pear_fresh                                  0.006308
apple_fresh                                 0.004966
olive_oil_cold_pressed_extra_virgin         0.000000
Length: 117, dtype: float64

In [68]:
# selecting high protein to calorie ratio foods
protein_foods = foods.loc[protein_calories.sort_values(ascending=False)[0:40].index,:]\
.query("calories > 50")

In [81]:
constraints.drop(['saturated_fat','cholesterol'])

,min,max
protein,56.0,160.0
fat_total,22.2,78.0
sodium,1500.0,2300.0
choline,550.0,3500.0
folate,400.0,1000.0
niacin,16.0,35.0
pantothenic_acid,5.0,NaN
vitamin_b2,1.1,NaN
vitamin_b1,1.2,NaN
vitamin_a,900.0,3000.0


In [100]:
necessary_nutrients = [c for c in constraints.index if c not in ['saturated_fat','cholesterol']]

In [98]:
constraints.loc[~constraints.index.isin(['saturated_fat','cholesterol']),:].index.values

array(['protein', 'fat_total', 'sodium', 'choline', 'folate', 'niacin',
       'pantothenic_acid', 'vitamin_b2', 'vitamin_b1', 'vitamin_a',
       'vitamin_b12', 'vitamin_b6', 'vitamin_c', 'vitamin_d_mcg',
       'vitamin_k', 'calcium', 'copper', 'iron', 'magnesium', 'manganese',
       'phosphorus', 'potassium', 'selenium', 'zinc', 'carbohydrates',
       'fiber', 'molybdenum', 'chloride', 'biotin', 'chromium',
       'cysteine_methionine', 'histidine', 'leucine', 'lysine',
       'methionine', 'phenylalanine_tyrosine', 'threonine', 'valine'],
      dtype=object)

In [112]:
foods[necessary_nutrients].idxmax()

protein                   chicken_pasture-raised_breast_roasted
fat_total                      lamb_grass-fed_lean_loin_roasted
sodium                                     shrimp_large_steamed
choline                                    shrimp_large_steamed
folate                                           lentils_cooked
niacin                              tuna_yellowfin_fillet_baked
pantothenic_acid                      mushrooms_shiitake_cooked
vitamin_b2                                      soybeans_cooked
vitamin_b1                                sunflower_seeds_dried
vitamin_a                                    sweet_potato_baked
vitamin_b12                            sardines_atlantic_canned
vitamin_b6                          tuna_yellowfin_fillet_baked
vitamin_c                                          papaya_fresh
vitamin_d_mcg                          salmon_wild_coho_broiled
vitamin_k                                           kale_cooked
calcium                                 

In [110]:
max_foods = foods.loc[list(set(foods[necessary_nutrients].idxmax().values)),:]

In [113]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in max_foods.index],
}

data_foods.update({col: protein_foods[col] for col in protein_foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Integer')

# Define the objective function to minimize the total number of servings
problem += lpSum(food_vars[food] for food in df_foods['Food'])

# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/e9e809cb903c46ec9827f28607c605b7-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/e9e809cb903c46ec9827f28607c605b7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 65 COLUMNS
At line 1353 RHS
At line 1414 BOUNDS
At line 1439 ENDATA
Problem MODEL has 60 rows, 24 columns and 1215 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: -1
No solution found.


Next I will solve to minimize calories since minimizing for serving size has no solution.

In [71]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
problem += lpSum(food_vars[food] for food in df_foods['Food'])

# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/33ae857b3ae640098857e51e295e6eb7-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/33ae857b3ae640098857e51e295e6eb7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 67 COLUMNS
At line 6185 RHS
At line 6248 BOUNDS
At line 6249 ENDATA
Problem MODEL has 62 rows, 117 columns and 6000 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
Perturbing problem by 0.001% of 43.417997 - largest nonzero change 9.3478336e-05 ( 0.039207549%) - largest zero change 0
0  Obj 0 Primal inf 183710.73 (39)
7  Obj 109764.51 Prim

Looking to minimize calories:

In [72]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
ae_calories = LpAffineExpression([(food_vars[food],df_foods.loc[food,'calories']) for food in df_foods['Food']])
problem += ae_calories
# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/59c0c3fe77e54ab292ad54701f639071-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/59c0c3fe77e54ab292ad54701f639071-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 67 COLUMNS
At line 6185 RHS
At line 6248 BOUNDS
At line 6249 ENDATA
Problem MODEL has 62 rows, 117 columns and 6000 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 183710.73 (39)
11  Obj 35420182 Primal inf 19078295 (25)
Primal infeasible - objective value 35420182
PrimalInfeasible objective 35420182.37 - 11 iterations 

In [73]:
constraints.drop(index=['calories'],inplace=True)
# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
ae_calories = LpAffineExpression([(food_vars[food],df_foods.loc[food,'calories']) for food in df_foods['Food']])
problem += ae_calories
# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/a4a77055d0ac412b9ca6d7f3d0a79e87-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/a4a77055d0ac412b9ca6d7f3d0a79e87-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 65 COLUMNS
At line 5949 RHS
At line 6010 BOUNDS
At line 6011 ENDATA
Problem MODEL has 60 rows, 117 columns and 5766 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 183679.88 (38)
11  Obj 35420182 Primal inf 18349995 (24)
Primal infeasible - objective value 35420182
PrimalInfeasible objective 35420182.37 - 11 iterations 